### Итоговый просмотр результатов по финальному этапу улучшения RAG

In [1]:
import pandas as pd
import numpy as np

# Пути к CSV файлам
csv_paths = [
    '../eval_base_rag/evaluete_base_rag.csv',
    '../eval_base_rag/evaluete_base_rag_gpt.csv',
    '../eval_advanced_rag/html_split/eval_Advanced_RAG_1_gigachat.csv',
    '../eval_advanced_rag/html_split/eval_Advanced_RAG_1_gpt_3.5.csv',
    '../eval_advanced_rag/html_split/eval_Advanced_RAG_2_gpt_3.5.csv',
    '../eval_advanced_rag/html_split/eval_Advanced_RAG_3_gpt_3.5.csv',
    '../eval_advanced_rag/html_split/eval_Advanced_RAG_4_gpt_3.5.csv',
    '../eval_advanced_rag/html_split/eval_Advanced_RAG_5_gpt_3.5.csv',
    '../eval_advanced_rag/html_split/eval_Advanced_RAG_6_gpt_3.5.csv'
]

# Массив с названиями методов (или столбцов)
method_names = [
    'Base rag', 
    'Base rag gpt', 
    'RAG+_1_gigachat',
    'RAG+_1_gpt_3.5',
    'RAG+_2_gpt_3.5',
    'RAG+_3_gpt_3.5',
    'RAG+_4_gpt_3.5',
    'RAG+_5_gpt_3.5',
    'RAG+_6_gpt_3.5'
]

# Столбцы, которые нужно пропустить при вычислении средних значений
columns_to_skip = ['question', 'contexts', 'ground_truth', 'answer']

# Инициализация пустого списка для хранения данных
data = []

# Массив для хранения всех уникальных метрик
all_metrics = set()

# Обработка каждого CSV файла из массива путей
for method_index, csv_path in enumerate(csv_paths):
    df = pd.read_csv(csv_path)
    
    # Вычисление средних значений для всех остальных столбцов
    mean_values = df.drop(columns=columns_to_skip).mean()
    
    # Преобразование средних значений в словарь
    mean_dict = mean_values.to_dict()
    
    # Добавление метрик в набор данных
    method_data = {key: float(value) for key, value in mean_dict.items()}
    all_metrics.update(method_data.keys())
    data.append(pd.Series(method_data, name=method_names[method_index]))

# Создание DataFrame из собранных данных
df_metrics = pd.concat(data, axis=1)

# Убедимся, что все метрики присутствуют в каждом столбце
for metric in all_metrics:
    if metric not in df_metrics.index:
        df_metrics.loc[metric] = np.nan  # Используем NaN для отсутствующих значений

# Заполнение отсутствующих значений NaN (замена '-' на NaN)
df_metrics = df_metrics.apply(pd.to_numeric, errors='coerce')

# Сохранение DataFrame в CSV файл
df_metrics.to_csv('metrics_step_advanced_RAG.csv')

print("Таблица метрик сохранена в файл metrics_step_advanced_RAG.csv")

Таблица метрик сохранена в файл metrics_step_advanced_RAG.csv


In [ ]:
df_metrics

,Base rag,Base rag gpt,RAG+_1_gigachat,RAG+_1_gpt_3.5,RAG+_2_gpt_3.5,RAG+_3_gpt_3.5,RAG+_4_gpt_3.5,RAG+_5_gpt_3.5,RAG+_6_gpt_3.5
faithfulness,0.839286,0.943548,0.925521,0.934930,0.815854,0.854422,0.716726,0.877778,0.854735
answer_relevancy,0.711729,0.819250,0.697932,0.832147,0.792119,0.848508,0.751150,0.842827,0.839272
context_precision,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
context_relevancy,0.754918,0.750033,0.780206,0.767275,0.764401,0.781642,0.783798,0.792418,0.798884
answer_similarity,0.923925,0.923702,0.914838,0.927357,0.925636,0.928973,0.920985,0.929259,0.926247
bleu_score,0.093522,0.060766,0.101752,0.066095,0.067560,0.064071,0.060195,0.070250,0.068312
sim-spacy,0.857789,0.849192,0.848899,0.874437,0.882084,0.887738,0.868104,0.898265,0.886603
cos-sim-TF-IDF,0.265224,0.239637,0.307029,0.269840,0.274250,0.296808,0.269838,0.290073,0.283390
cos-sim-BertModel,0.982229,0.981884,0.979668,0.980829,0.980588,0.982798,0.979693,0.982743,0.982738


In [ ]:
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: #d3d3d3' if v else '' for v in is_max]

# Применение стиля к DataFrame
styled_df = df_metrics.style.apply(highlight_max, axis=1)
styled_df

,Base rag,Base rag gpt,RAG+_1_gigachat,RAG+_1_gpt_3.5,RAG+_2_gpt_3.5,RAG+_3_gpt_3.5,RAG+_4_gpt_3.5,RAG+_5_gpt_3.5,RAG+_6_gpt_3.5
faithfulness,0.839286,0.943548,0.925521,0.934930,0.815854,0.854422,0.716726,0.877778,0.854735
answer_relevancy,0.711729,0.819250,0.697932,0.832147,0.792119,0.848508,0.751150,0.842827,0.839272
context_precision,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
context_relevancy,0.754918,0.750033,0.780206,0.767275,0.764401,0.781642,0.783798,0.792418,0.798884
answer_similarity,0.923925,0.923702,0.914838,0.927357,0.925636,0.928973,0.920985,0.929259,0.926247
bleu_score,0.093522,0.060766,0.101752,0.066095,0.067560,0.064071,0.060195,0.070250,0.068312
sim-spacy,0.857789,0.849192,0.848899,0.874437,0.882084,0.887738,0.868104,0.898265,0.886603
cos-sim-TF-IDF,0.265224,0.239637,0.307029,0.269840,0.274250,0.296808,0.269838,0.290073,0.283390
cos-sim-BertModel,0.982229,0.981884,0.979668,0.980829,0.980588,0.982798,0.979693,0.982743,0.982738


'RAG my smart 100-220 token' и 'RAG smart 1000 / 100' имеют примерно одинаковое разбиение по кускам

### RAGAS

**Faithfulness**
- Насколько фактически точен сгенерированный ответ.
- Это измеряет фактическую согласованность сгенерированного ответа с данным контекстом. Он рассчитывается на основе ответа и полученного контекста. Ответ масштабируется до (0,1) диапазона. Чем выше, тем лучше.
Сгенерированный ответ считается верным, если все утверждения, сделанные в ответе, могут быть выведены из данного контекста. Чтобы рассчитать это, сначала определяется набор утверждений из сгенерированного ответа. Затем каждое из этих утверждений проверяется с учетом данного контекста, чтобы определить, можно ли вывести его из данного контекста или нет. Оценка верности дается по делению на
  
**Answer Relevancy**
- Насколько релевантен сгенерированный ответ на вопрос (насколько ответы соответствуют заданным вопросам).
  
**Context Relevancy**
- Метрика оценивает релевантность извлеченного контекста на основе вопроса и контекста.
  
**Answer Semantic Similarity**
- Метрика оценивает семантическое сходство между сгенерированным ответом и эталонным ответом.

### Другие метрики

**BLEU Score**
- BLEU (Bilingual Evaluation Understudy) — это метрика, используемая для оценки качества машинного перевода или сгенерированного текста. Она сравнивает машинно-сгенерированный текст с одним или несколькими эталонными текстами.
  
**Sim-SpaCy**
- Эта метрика оценивает сходство между сгенерированным ответом и эталонным ответом с использованием модели SpaCy. Она измеряет семантическое сходство текстов на основе их векторных представлений.
  
**Cos-Sim-TF-IDF**
- Метрика cosine similarity с использованием TF-IDF оценивает сходство между двумя текстами, измеряя угол между их векторными представлениями в пространстве признаков.
  
 **Cos-Sim-BertModel**
- Эта метрика оценивает семантическое сходство между сгенерированным ответом и эталонным ответом, используя предварительно обученную модель BERT. Она основывается на вычислении косинусного сходства между векторными представлениями текстов, полученными из BERT.
  
 **Human**
- Метрика считается вручную, где 1 - это идеальное совпадение ответа и правильного ответа, 0 - нет свопадения, 0.5 - частичное совпадение. И в конце суммируется и делится на общее количество.